### RatB1 recording info
- This is Utku's old animal NSD session where he recorded for 24 hours.
- There is a 3 hour disconnection/break in the dark cycle

In [ ]:
import numpy as np
import subjects
import matplotlib.pyplot as plt
from neuropy import analyses, plotting

sess = subjects.NsdOld().ratB1_2022_06_24[0]
print(sess.recinfo)


### Set probe configuration
- Two masmanidis probes of 128 ch each implanted bilaterally.
- x spacing = 22 um, y spacing = 13 um


In [ ]:
from neuropy.core import Shank, Probe, ProbeGroup
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
badchans = sess.recinfo.skipped_channels

# --- Masmanidis 4 shanks -----------
for i in range(4):
    chans = channel_groups[i]
    shank = Shank.auto_generate(
        columns=3,
        contacts_per_column=[10, 12, 10],
        xpitch=22,
        ypitch=13,
        y_shift_per_column=[0, -13, 0],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)

probe1 = Probe(shanks)

# --- Masmanidis 4 shanks -----------
shanks = []
for i in range(4, 8):
    chans = channel_groups[i]
    shank = Shank.auto_generate(
        columns=3,
        contacts_per_column=[10, 12, 10],
        xpitch=22,
        ypitch=13,
        y_shift_per_column=[0, -13, 0],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)

probe2 = Probe(shanks)
probe2.move((probe1.x_max + 500, 0))

prbgrp = ProbeGroup()
prbgrp.add_probe(probe1)
prbgrp.add_probe(probe2)

prbgrp.save(sess.filePrefix.with_suffix(".probegroup.npy"))
plot_probe(prbgrp)


### Writing probegroup to json format for spyking-circus 

In [ ]:
from neuropy.utils import probe_util

file = sess.filePrefix.with_suffix(".prb")
probe_util.write_spyking_circus(file, sess.probegroup, combine_shanks=True)


### Experimental paradigm
- Recording had a disconnection of around 3 hour in the dark cycle.
- So total recording duration is around 20 hours.

In [ ]:
import pandas as pd
from neuropy.core import Epoch

datetime_data = pd.read_csv(sess.filePrefix.with_suffix(".datetime.csv"))
durations = datetime_data.nFrames / sess.recinfo.dat_sampling_rate
epochs = pd.DataFrame(
    {
        "start": [0, 28155, 31868, 49323],
        "stop": [28153, 31866, 74707, 52828],
        "label": ["pre", "maze", "post", "remaze"],
    }
)

paradigm = Epoch(epochs)
paradigm.save(sess.filePrefix.with_suffix(".paradigm.npy"))


### Artifacts epochs

In [ ]:
signal = sess.eegfile.get_signal([20, 170])
artifact_epochs = analyses.detect_artifact_epochs(signal, thresh=8, edge_cutoff=2)
artifact_epochs.save(sess.filePrefix.with_suffix(".artifact.npy"))


In [ ]:
signal = sess.eegfile.get_signal([20])
plotting.plot_artifact_epochs(artifact_epochs, signal)


In [ ]:
from neuropy.io import SpykingCircusIO

file = sess.filePrefix.with_suffix(".dead")
SpykingCircusIO.write_epochs(file, sess.artifact)


### Importing spiketrains from Phy

#### Single folder

In [ ]:
from neuropy.io import PhyIO
from neuropy.core import Neurons
from pathlib import Path
import numpy as np

cluster_path = Path(
    "/data/Clustering/sessions/UtkuOldAnimals/RatB1/RatB1_2022-06-24_NSD_CA1_24Hrs/2022-06-24_NSD_CA1_24hrscrs-merged_cleaned.GUI"
)
chan_grps = sess.recinfo.channel_groups
phy_data = PhyIO(cluster_path)
spiketrains = phy_data.spiketrains
peak_chans = phy_data.peak_channels
waveforms = phy_data.waveforms
shank_id = sess.probegroup.get_shank_id_for_channels(peak_chans)

# neuron_type_id = phy_data.cluster_info.q.values
# neuron_type = np.ones(len(neuron_type_id), dtype="U5")
# neuron_type[neuron_type_id < 4] = "pyr"
# neuron_type[neuron_type_id == 6] = "mua"
# neuron_type[neuron_type_id == 8] = "inter"


neurons = Neurons(
    np.array(spiketrains, dtype=object),
    t_stop=sess.eegfile.duration,
    sampling_rate=phy_data.sampling_rate,
    peak_channels=peak_chans,
    waveforms=np.array(waveforms, dtype="object"),
    # neuron_type=neurons_type[0],
    shank_ids=np.array(shank_id).astype(int),
    metadata={"cluster_path": str(cluster_path)},
)

neurons.save(sess.filePrefix.with_suffix(".neurons"))


### Auto label neuron_type

In [ ]:
from neuropy.utils.neurons_util import estimate_neuron_type, calculate_neurons_acg

neurons_type = estimate_neuron_type(sess.neurons)
# acgs = calculate_neurons_acg(neurons,window_size=0.1,plot=True)


In [ ]:
from neuropy.core import Neurons

neurons_new = Neurons(
    spiketrains=sess.neurons.spiketrains,
    sampling_rate=30000,
    t_stop=sess.neurons.t_stop,
    waveforms=sess.neurons.waveforms,
    neuron_type=neurons_type[0],
    shank_ids=sess.neurons.shank_ids,
)

neurons_new.save(sess.filePrefix.with_suffix(".neurons"))


In [ ]:
neurons_new.shank_ids.shape


In [ ]:
from neuropy.utils.neurons_util import acg_fit, acg_no_burst_fit
from scipy.optimize import curve_fit
from neuropy.utils.neurons_util import calculate_neurons_acg


neurons = sess.neurons

# Mua selection
n_spikes = neurons.n_spikes
isi = neurons.get_isi()
ref_nspikes = isi[:, :2].sum(axis=1)
violations = ref_nspikes * 100 / n_spikes
mua_indx = violations > 1

# too few spikes
low_nspikes_indx = n_spikes < 1000


bad_indx = np.logical_or(mua_indx, low_nspikes_indx)

good_neurons = neurons[~bad_indx]
good_nspikes = n_spikes[~bad_indx]

acgs = calculate_neurons_acg(neurons, bin_size=0.0005, window_size=0.1)
acgs[:, 100:105] = 0
acgs = acgs / neurons.n_spikes.reshape(-1, 1) / 0.005
t = np.arange(0.5, 50.5, 0.5)

p_initial = [20, 1, 30, 2, 0.5, 5, 1.5, 0]

params = []
for i, y in enumerate(acgs):
    lb = np.array([1, 0.1, -5, 0, -100, 0, 0.1, 0])
    ub = np.array([500, 50, 500, 25, 70, 20, 5, 100])

    try:
        popt, pcov = curve_fit(
            acg_no_burst_fit, t, y[101:], p0=p_initial[:-2], bounds=(lb[:-2], ub[:-2])
        )
    except:
        popt, pcov = curve_fit(acg_fit, t, y[101:], p0=p_initial, bounds=(lb, ub))
        # popt = np.concatenate([popt, [0, 0]])

    params.append(popt)

# params = np.array(params)
# _, ax = plt.subplots()
# ax.plot(t, y)
# # ax.plot(t,np.exp(-(t-popt[5])/popt[0]))
# ax.plot(t, acg_fit(t, *popt))


In [ ]:
from neuropy.utils.neurons_util import calculate_neurons_acg

neurons = sess.neurons.get_neuron_type("pyr")
acgs = calculate_neurons_acg(neurons)

_, axs = plt.subplots(12, 11)
axs = axs.reshape(-1)

for i in range(neurons.n_neurons):
    axs[i].plot(acgs[i])


In [ ]:
_, ax = plt.subplots()

ax.plot(acgs.max(axis=1), ".")
ax.axhline(1)


In [ ]:
_, axs = plt.subplots(4, 4)

low_acg = acgs[acgs.max(axis=1) < 8, :]
axs = axs.reshape(-1)

for i in range(16):
    axs[i].plot(low_acg[i])
    axs[i].set_title(low_acg[i].max())


In [ ]:
np.vstack((np.arange(5), np.arange(5)))


In [ ]:
_, ax = plt.subplots()

# ax.plot(params[:, 0], params[:, 2], ".")
ax.plot(t, y[101:])
# ax.plot(t, acg_fit(t, *popt))


In [ ]:
_, axs = plt.subplots(11, 11, sharex=True)
axs = axs.reshape(-1)

for i, p in enumerate(params):
    axs[i].plot(t, acgs[i][101:])
    if len(p) < 8:
        axs[i].plot(t, acg_no_burst_fit(t, *p))
    else:
        axs[i].plot(t, acg_fit(t, *p))


In [ ]:
_, ax = plt.subplots()

a = [_[0] for _ in params]
b = [_[1] for _ in params]

ax.scatter(a, b)
ax.set_xscale("log")
# ax.set_yscale('log')


In [ ]:
_, axs = plt.subplots(11, 11)
axs = axs.reshape(-1)


acgs = calculate_neurons_acg()
for i in range(len(good_neurons)):
    axs[i].plot(t, acgs[i][101:])
    axs[i].plot(t, acg_fit(t, *params[i]))


In [ ]:
params.shape


### Create MUA

In [ ]:
mua = sess.neurons.get_mua()
mua.save(sess.filePrefix.with_suffix(".mua"))


### Position data

In [ ]:
from neuropy.io import OptitrackIO
from neuropy.core import Position
from pathlib import Path

opti_folder = sess.filePrefix.parent / "position"
opti_data = OptitrackIO(dirname=opti_folder, scale_factor=0.25)


In [ ]:
import pandas as pd
from datetime import datetime

# ------- maze align corection ---------
t_error = [0] * 9

# ---- startimes of concatenated .dat files
tracking_sRate = opti_data.sampling_rate
rec_datetime = pd.read_csv(sess.filePrefix.with_suffix(".datetime.csv"))
data_time = []
for i, file_time in enumerate(rec_datetime["StartTime"]):
    # sync_time = rec_datetime["sync_nframes"][i] / rec_datetime["sync_rate"][i]
    tbegin = datetime.strptime(file_time, "%Y-%m-%d_%H-%M-%S") + pd.Timedelta(
        t_error[i], unit="sec"
    )
    nframes = rec_datetime["nFrames"][i]
    duration = pd.Timedelta(nframes / sess.recinfo.dat_sampling_rate, unit="sec")
    tend = tbegin + duration
    trange = pd.date_range(
        start=tbegin,
        end=tend,
        periods=int(duration.total_seconds() * tracking_sRate),
        inclusive="left",
    )
    data_time.extend(trange)
data_time = pd.to_datetime(data_time)


x, y, z = opti_data.get_position_at_datetimes(data_time)
traces = np.vstack((z, x, y))

position = Position(traces=traces, t_start=0, sampling_rate=opti_data.sampling_rate)
position.save(sess.filePrefix.with_suffix(".position.npy"))


_, ax = plt.subplots()
ax.plot(position.time, position.x)


In [ ]:
ax.plot(position.x, position.y)


### Linearize position

In [ ]:
from neuropy.utils import position_util

for e in ["maze1", "maze2"]:
    maze = sess.paradigm[e].flatten()
    maze_pos = sess.position.time_slice(maze[0], maze[1])
    linear_pos = position_util.linearize_position(maze_pos)
    # e = e.replace('-','')
    linear_pos.save(sess.filePrefix.with_suffix(f".{e}.linear"))


In [ ]:
_, ax = plt.subplots()
ax.plot(sess.maze1.time, sess.maze1.x)
ax.plot(sess.maze2.time, sess.maze2.x)
